In [ ]:
import tensorflow as tf
from preprocess import get_data
from models import GenomeMLP
import hyperparameters as hp
import numpy as np
import shap
import pickle
import scanpy as sc


from sklearn.calibration import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, f1_score
import pandas as pd

2025-01-20 11:58:37.628635: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-20 11:58:38.478991: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
adata = sc.read_h5ad("../neurons.h5ad")

label_encoder = LabelEncoder()

# encode the data

adata.obs['encoded_condition'] = label_encoder.fit_transform(adata.obs['condition'])


#loading in explainer and all shap values
with open('train_data.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open('train_labels.pkl', 'rb') as f:
    obs_train = pickle.load( f)

with open('validation_data.pkl', 'rb') as f:
    X_validate = pickle.load( f)
with open('validation_labels.pkl', 'rb') as f:
    obs_validate = pickle.load( f)

with open('test_data.pkl', 'rb') as f:
    X_test = pickle.load( f)
with open('test_labels.pkl', 'rb') as f:
    obs_test = pickle.load(f)

feature_names = adata.var.index.tolist()
# input = np.expand_dims(X_validate[0], axis = 0)



# union of top 50 of each cell type

In [5]:
import pandas as pd

cluster_decode = {
    0 : "Pyramidal layer2a/b",
    1 : "Pyramidal layer3",
    2 : "Deep Pyramidal",
    3 : "Semilunar",
    4 : "Inhibitory1",
    5 : "Inhibitory2",
    6 : "Contamination1",
    7 : "Contamination2",
    8 : "Pappa2"
}






# # Read in top 50 features for each cell type, find union of all top 50 features
# top_50df = pd.read_csv('top_100_features_new.csv')
# top_50_union = {}
# for i in X_test.obs['leiden'].unique():
#     print(f"Leiden cluster: {i}")
#     cluster_samples = X_test[X_test.obs['leiden'] == i]
#     # print(cluster_samples)
#     for j in range(len(top_50df[cluster_decode[int(i)]])):
#         if j < 50:
#             if i not in top_50_union:
#                 top_50_union[top_50df[cluster_decode[int(i)]][j]] = 0
#             top_50_union[top_50df[cluster_decode[int(i)]][j]] += 1
#         else: 
#             continue
# union50 = sorted(list(top_50_union.keys()), reverse = True,  key = lambda x : top_50_union[x])

# union50csv = pd.DataFrame()

# union50csv['union50'] = union50


# union50csv.to_csv('top_101_features.csv', index=False)
# union50 = union50[:50]

cluster_decode = {
    0 : "Pyramidal layer2a/b",
    1 : "Pyramidal layer3",
    2 : "Deep Pyramidal",
    3 : "Semilunar",
    4 : "Inhibitory1",
    5 : "Inhibitory2",
    6 : "Contamination1",
    7 : "Contamination2",
    8 : "Pappa2"
}
# Step 1: Read the CSV file into a Pandas DataFrame
df = pd.read_csv('top_100_features_new.csv')
df = df[[cluster_decode[int(x)] for x in X_test.obs['leiden'].unique()]]
df = df.head(50)

# Step 2: Combine all columns into one mega column (flattening)
mega_column = pd.Series(df.values.ravel()).drop_duplicates().reset_index(drop=True)

# # Step 3: Remove duplicates and drop NaN values (if any)
mega_column_unique = mega_column.drop_duplicates().dropna()

# Optionally convert it back into a DataFrame with one column
df_mega = pd.DataFrame(mega_column_unique, columns=['Mega_Column'])

# Step 4: Save or print the result

df_mega.to_csv('output.csv', index=False)
union50= list(mega_column)


# calculating variance for each gene

In [9]:
average_expressions_df = pd.DataFrame()

for i in range(6):
    i = str(i)
    cluster_samples = X_test[X_test.obs['leiden'] == i]
    cluster_values = []
    
    for gene in union50:
        gene_index = None
        if gene in cluster_samples.var.index:
            gene_index = cluster_samples.var.index.get_loc(gene)
        
        if gene_index is None:
            cluster_values.append(0)
            continue

        gene_expression_values = cluster_samples.raw[:, gene_index].X
        mean_expression = np.mean(gene_expression_values)
        
        cluster_values.append(mean_expression)

    column_name = cluster_decode[int(i)]
    average_expressions_df[column_name] = cluster_values





In [11]:
average_expressions_df["genes"] = union50

In [12]:
average_expressions_df

Pyramidal layer2a/b  Pyramidal layer3  Deep Pyramidal  Semilunar  \
0               1.457085          1.274118        1.278410   1.488933   
1               1.241385          1.445434        1.533076   1.151417   
2               1.310891          1.192042        1.192261   1.381200   
3               1.341394          1.256524        1.225554   1.372433   
4               1.253611          1.190644        1.230164   1.355801   
..                   ...               ...             ...        ...   
136             0.436097          0.460586        0.459360   0.470145   
137             0.044256          0.041382        0.045678   0.043898   
138             0.356930          0.225411        0.164857   0.252338   
139             1.810298          2.119592        1.692187   1.629838   
140             1.528397          1.728505        1.396783   0.970839   

     Inhibitory1  Inhibitory2       genes  
0       1.495970     1.428339    AY036118  
1       1.258538     1.224327  AC149090.1  
2       1.340165     1.316278       Lars2  
3       1.388567     1.424761        Cdk8  
4       1.324957     1.301368       Cmss1  
..           ...          ...         ...  
136     0.585174     0.499711         Uty  
137     0.043874     0.037075       Wdr72  
138     0.151721     0.161196       Tmem9  
139     1.438928     1.540896      March1  
140     1.536094     1.301140       Ntrk3  

[141 rows x 7 columns]

In [31]:
var = []
for _, row in average_expressions_df.iterrows():
    var.append(np.var(list(row[0:6])))

In [32]:
average_expressions_df["var"] = var

In [34]:
average_expressions_df.to_csv("mean_variance_expressions.csv")